In [5]:
import numpy as np
import pandas as pd

# Define a standard DES S-box (example: S-box 1 from DES)
S_BOX = [
    [14, 4, 13, 1, 2, 15, 11, 8, 3, 10, 6, 12, 5, 9, 0, 7],
    [0, 15, 7, 4, 14, 2, 13, 1, 10, 6, 12, 11, 9, 5, 3, 8],
    [4, 1, 14, 8, 13, 6, 2, 11, 15, 12, 9, 7, 3, 10, 5, 0],
    [15, 12, 8, 2, 4, 9, 1, 7, 5, 11, 3, 14, 10, 0, 6, 13]
]

# Function to apply S-box transformation
def apply_sbox(x):
    row = ((x & 0b100000) >> 4) | (x & 0b000001)  # First and last bit
    col = (x & 0b011110) >> 1  # Middle 4 bits
    return S_BOX[row][col]

# Generate DES S-box mappings
sbox_results = [(format(x, '06b'), format(apply_sbox(x), '04b')) for x in range(64)]

# Define an approximation function S_k using a polynomial map
np.random.seed(42)  # Fix randomness for reproducibility
A = np.random.randint(1, 16, size=64)  # Random coefficients
B = np.random.randint(0, 16, size=64)
degree = 3  # Chosen polynomial degree

def approx_sbox(x, A, B, degree):
    return (A[x] * (x ** degree) + B[x]) % 16  # Ensuring 4-bit output

# Generate approximated S-box outputs
approx_results = [(format(x, '06b'), format(approx_sbox(x, A, B, degree), '04b')) for x in range(64)]

# Print tables in well-formatted way
print("DES S-Box Mapping:")
print("+--------+--------------+")
print("| Input  | S-Box Output |")
print("+--------+--------------+")
for inp, out in sbox_results:
    print(f"| {inp} |      {out}       |")
print("+--------+--------------+")

print("\nApproximated S_k Mapping:")
print("+--------+------------------+")
print("| Input  | Approx S_k Output |")
print("+--------+------------------+")
for inp, out in approx_results:
    print(f"| {inp} |        {out}        |")
print("+--------+------------------+")

DES S-Box Mapping:
+--------+--------------+
| Input  | S-Box Output |
+--------+--------------+
| 000000 |      1110       |
| 000001 |      0000       |
| 000010 |      0100       |
| 000011 |      1111       |
| 000100 |      1101       |
| 000101 |      0111       |
| 000110 |      0001       |
| 000111 |      0100       |
| 001000 |      0010       |
| 001001 |      1110       |
| 001010 |      1111       |
| 001011 |      0010       |
| 001100 |      1011       |
| 001101 |      1101       |
| 001110 |      1000       |
| 001111 |      0001       |
| 010000 |      0011       |
| 010001 |      1010       |
| 010010 |      1010       |
| 010011 |      0110       |
| 010100 |      0110       |
| 010101 |      1100       |
| 010110 |      1100       |
| 010111 |      1011       |
| 011000 |      0101       |
| 011001 |      1001       |
| 011010 |      1001       |
| 011011 |      0101       |
| 011100 |      0000       |
| 011101 |      0011       |
| 011110 |      0111       |
| 01

In [6]:
import numpy as np

# 1) Define a standard DES S-box (example: S-box 1 from DES)
S_BOX = [
    [14, 4, 13, 1, 2, 15, 11, 8, 3, 10, 6, 12, 5, 9, 0, 7],
    [0, 15, 7, 4, 14, 2, 13, 1, 10, 6, 12, 11, 9, 5, 3, 8],
    [4, 1, 14, 8, 13, 6, 2, 11, 15, 12, 9, 7, 3, 10, 5, 0],
    [15, 12, 8, 2, 4, 9, 1, 7, 5, 11, 3, 14, 10, 0, 6, 13]
]

# 2) Function to apply S-box transformation
def apply_sbox(x):
    """
    x: integer in [0, 63] (6 bits)
    Returns the corresponding 4-bit output from S_BOX[ row ][ col ].
    row = bits (x5 x0)
    col = bits (x4 x3 x2 x1)
    """
    row = ((x & 0b100000) >> 4) | (x & 0b000001)
    col = (x & 0b011110) >> 1
    return S_BOX[row][col]

# 3) We want an "approx_sbox" of the form:
#        approx_sbox(x) = (A[x] * x^degree + B[x]) mod 16
# such that it exactly matches apply_sbox(x) for x in [0..63].

degree = 3  # can be any chosen polynomial degree, we'll keep it as 3

# Initialize arrays A and B of length 64
A = np.zeros(64, dtype=np.int32)
B = np.zeros(64, dtype=np.int32)

# Fill A[x], B[x] so that (A[x] * x^degree + B[x]) mod 16 = S_BOX(x)
for x in range(64):
    desired = apply_sbox(x)                # The correct S-box output
    x_degree_mod16 = pow(x, degree, 16)    # x^degree mod 16

    if x_degree_mod16 != 0:
        # Let A[x] = 1, so we need B[x] = (desired - x^degree) mod 16
        A[x] = 1
        B[x] = (desired - x_degree_mod16) % 16
    else:
        # Let A[x] = 0, so output is just B[x]; set B[x] = desired
        A[x] = 0
        B[x] = desired

def approx_sbox(x, A, B, degree):
    """ Polynomial-based approximation that is now an exact match. """
    return (A[x] * (x ** degree) + B[x]) % 16

# 4) Generate and compare outputs
sbox_results = [(format(x, '06b'), format(apply_sbox(x), '04b')) for x in range(64)]
approx_results = [(format(x, '06b'), format(approx_sbox(x, A, B, degree), '04b')) for x in range(64)]

# Check for mismatches
all_match = True
for x in range(64):
    real_val = apply_sbox(x)
    approx_val = approx_sbox(x, A, B, degree)
    if real_val != approx_val:
        print(f"Mismatch at x={x} (decimal): real={real_val}, approx={approx_val}")
        all_match = False

if all_match:
    print("SUCCESS: The polynomial-based S-box matches the real S-box for all 64 inputs.\n")

# 5) Print both mappings in a nice table
print("DES S-Box Mapping:")
print("+--------+--------------+")
print("| Input  | S-Box Output |")
print("+--------+--------------+")
for inp, out in sbox_results:
    print(f"| {inp} |      {out}       |")
print("+--------+--------------+")

print("\nApproximated S_k Mapping (Now Exact):")
print("+--------+------------------+")
print("| Input  | Approx S_k Output |")
print("+--------+------------------+")
for inp, out in approx_results:
    print(f"| {inp} |        {out}        |")
print("+--------+------------------+")


SUCCESS: The polynomial-based S-box matches the real S-box for all 64 inputs.

DES S-Box Mapping:
+--------+--------------+
| Input  | S-Box Output |
+--------+--------------+
| 000000 |      1110       |
| 000001 |      0000       |
| 000010 |      0100       |
| 000011 |      1111       |
| 000100 |      1101       |
| 000101 |      0111       |
| 000110 |      0001       |
| 000111 |      0100       |
| 001000 |      0010       |
| 001001 |      1110       |
| 001010 |      1111       |
| 001011 |      0010       |
| 001100 |      1011       |
| 001101 |      1101       |
| 001110 |      1000       |
| 001111 |      0001       |
| 010000 |      0011       |
| 010001 |      1010       |
| 010010 |      1010       |
| 010011 |      0110       |
| 010100 |      0110       |
| 010101 |      1100       |
| 010110 |      1100       |
| 010111 |      1011       |
| 011000 |      0101       |
| 011001 |      1001       |
| 011010 |      1001       |
| 011011 |      0101       |
| 011100 |  

In [9]:
import numpy as np

# 1) Define a standard DES S-box (example: S-box 1 from DES)
S_BOX = [
    [14, 4, 13, 1, 2, 15, 11, 8, 3, 10, 6, 12, 5, 9, 0, 7],
    [0, 15, 7, 4, 14, 2, 13, 1, 10, 6, 12, 11, 9, 5, 3, 8],
    [4, 1, 14, 8, 13, 6, 2, 11, 15, 12, 9, 7, 3, 10, 5, 0],
    [15, 12, 8, 2, 4, 9, 1, 7, 5, 11, 3, 14, 10, 0, 6, 13]
]

# 2) Function to apply S-box transformation
def apply_sbox(x):
    """
    x: integer in [0, 63] (6 bits)
    Returns the corresponding 4-bit output from S_BOX[row][col].
    row = bits (x5 x0)
    col = bits (x4 x3 x2 x1)
    """
    row = ((x & 0b100000) >> 4) | (x & 0b000001)  # uses top bit + bottom bit
    col = (x & 0b011110) >> 1
    return S_BOX[row][col]

# 3) We define a polynomial-based function:
#    approx_sbox(x) = (A[x] * x^degree + B[x]) % 16
# and want it to exactly match apply_sbox(x) for x in [0..63].

degree = 3  # chosen polynomial degree

# Initialize A and B as numpy arrays of length 64
A = np.zeros(64, dtype=np.int32)
B = np.zeros(64, dtype=np.int32)

# Fill A[x], B[x] so that (A[x]*x^degree + B[x]) mod 16 = S_BOX(x)
for x in range(64):
    desired = apply_sbox(x)               # correct S-box output
    x_degree_mod16 = pow(x, degree, 16)   # x^degree (mod 16)

    if x_degree_mod16 != 0:
        # let A[x] = 1, so we need B[x] = (desired - x^degree) mod 16
        A[x] = 1
        B[x] = (desired - x_degree_mod16) % 16
    else:
        # let A[x] = 0, so output is just B[x]; set B[x] = desired
        A[x] = 0
        B[x] = desired

def approx_sbox(x, A, B, degree):
    """ Polynomial-based approximation that is now an exact match. """
    return (A[x] * (x ** degree) + B[x]) % 16

# 4) Generate and compare outputs
sbox_results   = [(format(x, '06b'), format(apply_sbox(x),       '04b')) for x in range(64)]
approx_results = [(format(x, '06b'), format(approx_sbox(x, A, B, degree), '04b')) for x in range(64)]

# Check for mismatches
all_match = True
for x in range(64):
    real_val = apply_sbox(x)
    approx_val = approx_sbox(x, A, B, degree)
    if real_val != approx_val:
        print(f"Mismatch at x={x}: real={real_val}, approx={approx_val}")
        all_match = False

if all_match:
    print("SUCCESS: The polynomial-based S-box matches the real S-box for all 64 inputs.\n")

# 5) Print the A and B arrays
print("A array (decimal):")
print(A)
print("\nB array (decimal):")
print(B)
print()

# 6) Print both mappings side by side for visual check
print("DES S-Box Mapping:")
print("+--------+--------------+")
print("| Input  | S-Box Output |")
print("+--------+--------------+")
for inp, out in sbox_results:
    print(f"| {inp} |      {out}       |")
print("+--------+--------------+")

print("\nApproximated S_k Mapping (Now Exact):")
print("+--------+------------------+")
print("| Input  | Approx S_k Output |")
print("+--------+------------------+")
for inp, out in approx_results:
    print(f"| {inp} |        {out}        |")
print("+--------+------------------+")


SUCCESS: The polynomial-based S-box matches the real S-box for all 64 inputs.

A array (decimal):
[0 1 1 1 0 1 1 1 0 1 1 1 0 1 1 1 0 1 1 1 0 1 1 1 0 1 1 1 0 1 1 1 0 1 1 1 0
 1 1 1 0 1 1 1 0 1 1 1 0 1 1 1 0 1 1 1 0 1 1 1 0 1 1 1]

B array (decimal):
[14 15 12  4 13 10  9 13  2  5  7 15 11  8  0  2  3  9  2 11  6 15  4  4
  5  0  1  2  0 14 15  9  4 14  9  1 14 11  0 11 13 11 14  6  2 12  3  8
 15  4  4  0  9  6 15  7  3  1  2 13  5  1  8 14]

DES S-Box Mapping:
+--------+--------------+
| Input  | S-Box Output |
+--------+--------------+
| 000000 |      1110       |
| 000001 |      0000       |
| 000010 |      0100       |
| 000011 |      1111       |
| 000100 |      1101       |
| 000101 |      0111       |
| 000110 |      0001       |
| 000111 |      0100       |
| 001000 |      0010       |
| 001001 |      1110       |
| 001010 |      1111       |
| 001011 |      0010       |
| 001100 |      1011       |
| 001101 |      1101       |
| 001110 |      1000       |
| 001111 |      0001  

In [40]:
import numpy as np

# ------------------------------------------------------------------
# 1) Define DES S-box (S-box 1) and a helper to get its 4-bit output.
# ------------------------------------------------------------------
S_BOX = [
    [14,  4, 13,  1,  2, 15, 11,  8,  3, 10,  6, 12,  5,  9,  0,  7],
    [ 0, 15,  7,  4, 14,  2, 13,  1, 10,  6, 12, 11,  9,  5,  3,  8],
    [ 4,  1, 14,  8, 13,  6,  2, 11, 15, 12,  9,  7,  3, 10,  5,  0],
    [15, 12,  8,  2,  4,  9,  1,  7,  5, 11,  3, 14, 10,  0,  6, 13]
]

def apply_sbox_6to4(x_6bit):
    """
    x_6bit: integer in [0..63], representing 6 bits.
    Returns an integer in [0..15], representing 4 bits (the S-box output).
    """
    row = ((x_6bit & 0b100000) >> 4) | (x_6bit & 0b000001)  # bits (x5, x0)
    col = (x_6bit & 0b011110) >> 1
    return S_BOX[row][col]

def get_bit(value, bitpos):
    """Return the 'bitpos'-th bit of 'value', counting from LSB=0."""
    return (value >> bitpos) & 1

# ------------------------------------------------------------------
# 2) Prepare a list of the S-box outputs for x in [0..63],
#    and represent them in 4 bits (sbit[x][j] = j-th bit of S(x)).
# ------------------------------------------------------------------
sbit = np.zeros((64, 4), dtype=np.uint8)
for x in range(64):
    s_val = apply_sbox_6to4(x)  # 0..15
    # We'll store each of the 4 bits in sbit[x][0..3].
    for j in range(4):
        sbit[x, j] = get_bit(s_val, j)   # j=0 => LSB, j=3 => MSB

# ------------------------------------------------------------------
# 3) We'll do a brute-force(ish) search:
#    - B in [0..63] (i.e., 64 ways to choose the 6-bit offset).
#    - For each of the 4 output bits j, find the best 6-bit vector a_j
#      that minimizes mismatch with sbit[x,j] over x in [0..63].
#
#    This yields a single B (6 bits) plus 4 separate a_j (each 6 bits)
#    => effectively a 6x4 matrix A in GF(2).
# ------------------------------------------------------------------

def popcount_6(x):
    """Return the number of bits set in a 6-bit integer x."""
    # Python 3.10+ has int.bit_count(), but let's be explicit:
    return bin(x).count("1")

best_overall_mismatches = 999999
best_B   = 0
best_A   = np.zeros((4,), dtype=np.uint8)  # store 4 columns a_j as 6-bit integers

# We'll try every possible 6-bit offset B.
for B_candidate in range(64):
    # For each output bit j, we find the best "a_j" in [0..63].
    # We'll store the mismatch counts here.
    mismatch_for_bit = [0]*4
    chosen_a_for_bit = [0]*4

    # Precompute x' = x ^ B_candidate for all x:
    x_xor_B = [x ^ B_candidate for x in range(64)]

    for j in range(4):
        # We want to find a_j in [0..63] that best matches sbit[x,j] = a_j dot x'.
        best_mismatch_j = 999999
        best_a_j        = 0

        for a_candidate in range(64):
            # Count how many x in [0..63] yield a mismatch
            mismatch_count = 0
            for x in range(64):
                # dot-product in GF(2) is popcount(a & x') mod 2
                dot_val = popcount_6(a_candidate & x_xor_B[x]) % 2
                if dot_val != sbit[x, j]:
                    mismatch_count += 1

            if mismatch_count < best_mismatch_j:
                best_mismatch_j = mismatch_count
                best_a_j        = a_candidate

        mismatch_for_bit[j] = best_mismatch_j
        chosen_a_for_bit[j] = best_a_j

    total_mismatch_B = sum(mismatch_for_bit)
    if total_mismatch_B < best_overall_mismatches:
        best_overall_mismatches = total_mismatch_B
        best_B = B_candidate
        best_A = np.array(chosen_a_for_bit, dtype=np.uint8)

# ------------------------------------------------------------------
# 4) Having found the best (B, A), let's see how well it approximates
#    S(x). We'll measure the mismatch rate.
# ------------------------------------------------------------------
mismatch_count = 0
for x in range(64):
    # Real S-box output:
    real_val = apply_sbox_6to4(x)

    # Approx output = ( (x ^ B) * A ) in GF(2)
    x_prime = x ^ best_B
    # Build 4 bits:
    approx_bits = 0
    for j in range(4):
        # a_j dot x_prime in GF(2)
        a_j = best_A[j]
        dot_val = popcount_6(a_j & x_prime) % 2
        approx_bits |= (dot_val << j)

    if approx_bits != real_val:
        mismatch_count += 1

mismatch_rate = mismatch_count / 64.0 * 100
print("Best linear approximation found!")
print(f"  Offset B (6 bits)   = {best_B} (binary: {best_B:06b})")
print("  Matrix A (6x4), stored as 4 columns (each a_j is a 6-bit integer):")
for j in range(4):
    print(f"    a_{j} = {best_A[j]:06b}")
print(f"\n  Mismatch count = {mismatch_count} out of 64 inputs")
print(f"  Mismatch rate  = {mismatch_rate:.2f}%\n")

# ------------------------------------------------------------------
# 5) Let's print out how our final approximation compares per input x.
# ------------------------------------------------------------------
print("Compare real vs. approx for each x in [0..63]:")
print("+--------+--------------+-----------------+")
print("|  x(6)  | S-Box Output | Approx S Output |")
print("+--------+--------------+-----------------+")
for x in range(64):
    real_val   = apply_sbox_6to4(x)
    # Compute approx
    x_prime = x ^ best_B
    approx_bits = 0
    for j in range(4):
        a_j = best_A[j]
        dot_val = popcount_6(a_j & x_prime) % 2
        approx_bits |= (dot_val << j)
    print(f"| {x:06b} |   {real_val:04b} ({real_val:2d})  |   {approx_bits:04b} ({approx_bits:2d})   |")
print("+--------+--------------+-----------------+")


Best linear approximation found!
  Offset B (6 bits)   = 1 (binary: 000001)
  Matrix A (6x4), stored as 4 columns (each a_j is a 6-bit integer):
    a_0 = 111111
    a_1 = 111111
    a_2 = 011011
    a_3 = 111011

  Mismatch count = 52 out of 64 inputs
  Mismatch rate  = 81.25%

Compare real vs. approx for each x in [0..63]:
+--------+--------------+-----------------+
|  x(6)  | S-Box Output | Approx S Output |
+--------+--------------+-----------------+
| 000000 |   1110 (14)  |   1111 (15)   |
| 000001 |   0000 ( 0)  |   0000 ( 0)   |
| 000010 |   0100 ( 4)  |   0000 ( 0)   |
| 000011 |   1111 (15)  |   1111 (15)   |
| 000100 |   1101 (13)  |   1100 (12)   |
| 000101 |   0111 ( 7)  |   0011 ( 3)   |
| 000110 |   0001 ( 1)  |   0011 ( 3)   |
| 000111 |   0100 ( 4)  |   1100 (12)   |
| 001000 |   0010 ( 2)  |   0000 ( 0)   |
| 001001 |   1110 (14)  |   1111 (15)   |
| 001010 |   1111 (15)  |   1111 (15)   |
| 001011 |   0010 ( 2)  |   0000 ( 0)   |
| 001100 |   1011 (11)  |   0011 ( 3)

In [ ]:
import time

# ------------------------------------------------------------------
# 1) Define DES S-box (S-box 1) and a helper to get its 4-bit output.
# ------------------------------------------------------------------
S_BOX = [
    [14,  4, 13,  1,  2, 15, 11,  8,  3, 10,  6, 12,  5,  9,  0,  7],
    [ 0, 15,  7,  4, 14,  2, 13,  1, 10,  6, 12, 11,  9,  5,  3,  8],
    [ 4,  1, 14,  8, 13,  6,  2, 11, 15, 12,  9,  7,  3, 10,  5,  0],
    [15, 12,  8,  2,  4,  9,  1,  7,  5, 11,  3, 14, 10,  0,  6, 13]
]

def apply_sbox_6to4(x_6bit):
    """
    x_6bit: integer in [0..63], representing 6 bits.
    Returns an integer in [0..15], representing 4 bits (the S-box output).
    """
    row = ((x_6bit & 0b100000) >> 4) | (x_6bit & 0b000001)  # bits (x5, x0)
    col = (x_6bit & 0b011110) >> 1
    return S_BOX[row][col]

def popcount6(x):
    """Return number of bits set in x (only x up to 6 bits is used)."""
    # On Python 3.10+, you can do: return x.bit_count()
    return bin(x).count('1')

# ------------------------------------------------------------------
# 2) Precompute the S-box outputs for x in [0..63].
#    We store them as a 4-bit integer for easy comparison.
# ------------------------------------------------------------------
sbox_out = [0]*64
for x in range(64):
    sbox_out[x] = apply_sbox_6to4(x)  # 0..15

# ------------------------------------------------------------------
# 3) We'll do a brute-force over all 2^30 combos of (A,B).
#
#    * B is stored in the low 6 bits of a 30-bit integer.
#    * A is stored in the high 24 bits of that integer.
#
#    A is a 6x4 matrix in GF(2). We'll interpret it as 4 blocks of 6 bits:
#      A_col0 = (A >>  0) & 0x3F
#      A_col1 = (A >>  6) & 0x3F
#      A_col2 = (A >> 12) & 0x3F
#      A_col3 = (A >> 18) & 0x3F
#    Then for an input x, after XOR with B, we do dot-products:
#      output_bit_j = popcount6( A_colj & (x ^ B) ) % 2
# ------------------------------------------------------------------

best_mismatch    = 999999
best_combination = 0    # will store the entire 30-bit integer

start_time = time.time()

TOTAL_SPACE = 1 << 30  # 2^30

def mismatch_count_for(A24, B6):
    """
    Compute how many x in [0..63] mismatch if we use
    approxS(x) = ( (x ^ B6) dot A24 ) in GF(2).

    A24 is a 24-bit integer => 4 chunks of 6 bits each.
    B6  is a 6-bit integer.
    """
    mismatches = 0

    # Extract 4 columns from A24 (each 6 bits)
    A_col0 =  (A24 >>  0) & 0x3F
    A_col1 =  (A24 >>  6) & 0x3F
    A_col2 = (A24 >> 12) & 0x3F
    A_col3 = (A24 >> 18) & 0x3F

    for x in range(64):
        real_val = sbox_out[x]  # 4-bit integer
        xprime  = x ^ B6
        # Build approx 4-bit output
        out0 = popcount6(A_col0 & xprime) & 1
        out1 = popcount6(A_col1 & xprime) & 1
        out2 = popcount6(A_col2 & xprime) & 1
        out3 = popcount6(A_col3 & xprime) & 1
        approx_val = (out3 << 3) | (out2 << 2) | (out1 << 1) | out0

        if approx_val != real_val:
            mismatches += 1
            # Early exit if it's already worse than the global best
            # (optional optimization)
            if mismatches > best_mismatch:
                return mismatches

    return mismatches

idx = 0
REPORT_INTERVAL = 1_000_000  # print progress every million combos

for comb in range(TOTAL_SPACE):
    B6  =  comb        & 0x3F
    A24 = (comb >>  6) & 0xFFFFFF  # top 24 bits

    # Evaluate mismatch
    m = mismatch_count_for(A24, B6)
    if m < best_mismatch:
        best_mismatch = m
        best_combination = comb
        # We can print partial progress
        print(f"[UPDATE] new best mismatch = {m} with comb=0x{comb:X} (decimal={comb})")
        # If mismatch=0, perfect match (which is effectively impossible for a DES S-Box),
        # so we could break early if that ever happens.
        if m == 0:
            break

    idx += 1
    if (idx % REPORT_INTERVAL) == 0:
        elapsed = time.time() - start_time
        speed = idx / elapsed
        remaining = TOTAL_SPACE - idx
        est_remain_sec = remaining / speed if speed > 0 else 9999999
        print(f"Progress: {idx}/{TOTAL_SPACE} (~{idx*100./TOTAL_SPACE:.2f}%), "
              f"best_mismatch={best_mismatch}, speed ~{speed:,.0f} combos/s, "
              f"ETA ~{est_remain_sec/3600:.2f}h")

end_time = time.time()
seconds = end_time - start_time

# decode best_combination
B6_final  =  best_combination        & 0x3F
A24_final = (best_combination >>  6) & 0xFFFFFF

m = mismatch_count_for(A24_final, B6_final)

print("\nDONE!")
print(f"  Explored {idx} combos (out of {TOTAL_SPACE}) in {seconds:.1f} seconds.")
print(f"  best mismatch = {m} out of 64, comb = 0x{best_combination:X} (decimal={best_combination}).")

# Reconstruct the final matrix
A_col0 =  (A24_final >>  0) & 0x3F
A_col1 =  (A24_final >>  6) & 0x3F
A_col2 = (A24_final >> 12) & 0x3F
A_col3 = (A24_final >> 18) & 0x3F

print(f"  B = {B6_final:06b}")
print("  A columns (6 bits each):")
print(f"    col0 = {A_col0:06b}")
print(f"    col1 = {A_col1:06b}")
print(f"    col2 = {A_col2:06b}")
print(f"    col3 = {A_col3:06b}")


[UPDATE] new best mismatch = 60 with comb=0x0 (decimal=0)
[UPDATE] new best mismatch = 59 with comb=0x80 (decimal=128)
[UPDATE] new best mismatch = 57 with comb=0x141 (decimal=321)
[UPDATE] new best mismatch = 56 with comb=0xFC1 (decimal=4033)
[UPDATE] new best mismatch = 55 with comb=0x2143 (decimal=8515)
[UPDATE] new best mismatch = 53 with comb=0x2492 (decimal=9362)
[UPDATE] new best mismatch = 52 with comb=0x15151 (decimal=86353)
[UPDATE] new best mismatch = 51 with comb=0x992D3 (decimal=627411)
Progress: 1000000/1073741824 (~0.09%), best_mismatch=51, speed ~13,568 combos/s, ETA ~21.96h
[UPDATE] new best mismatch = 50 with comb=0x12BFD5 (decimal=1228757)
Progress: 2000000/1073741824 (~0.19%), best_mismatch=50, speed ~13,983 combos/s, ETA ~21.29h
Progress: 3000000/1073741824 (~0.28%), best_mismatch=50, speed ~14,156 combos/s, ETA ~21.01h
Progress: 4000000/1073741824 (~0.37%), best_mismatch=50, speed ~14,157 combos/s, ETA ~20.99h
Progress: 5000000/1073741824 (~0.47%), best_mismatch=5